**[Joining Data in SQL](https://www.datacamp.com/courses/joining-data-in-postgresql)**

**Course Description**

Now that you've learned the basics of SQL in our [Intro to SQL for Data Science](https://www.datacamp.com/courses/intro-to-sql-for-data-science) course, it's time to supercharge your queries using joins and relational set theory! In this course you'll learn all about the power of joining tables while exploring interesting features of countries and their cities throughout the world. You will master inner and outer joins, as well as self-joins, semi-joins, anti-joins and cross joins - fundamental tools in any PostgreSQL wizard's toolbox. You'll fear set theory no more, after learning all about unions, intersections, and except clauses through easy-to-understand diagrams and examples. Lastly, you'll be introduced to the challenging topic of subqueries. You will see a visual perspective to grasp the ideas throughout the course using the mediums of Venn diagrams and other linking illustrations.

**Datasets**

* [Countries](https://assets.datacamp.com/production/repositories/1069/datasets/578834f5908e3b2fa575429a287586d1eaeb2e54/countries2.zip)
* [Leaders](https://assets.datacamp.com/production/repositories/1069/datasets/5aba4b2d25e3025de97d9715a022f5c24b74f347/leaders2.zip)
* [Diagrams](https://assets.datacamp.com/production/repositories/1069/datasets/379b79c12b968edafe24e4bc02fae89d090a9490/diagrams.zip)

**Imports**

In [ ]:
from sqlalchemy import create_engine
from sqlalchemy import MetaData
from sqlalchemy import Table
from sqlalchemy import Column
from sqlalchemy import Integer, String
from sqlalchemy import inspect
import pandas as pd
from pprint import pprint as pp

**Pandas Configuration Options**

In [ ]:
pd.set_option('max_columns', 200)
pd.set_option('max_rows', 300)
pd.set_option('display.expand_frame_repr', True)

**PostgreSQL Connection**

In [ ]:
engine = create_engine('postgresql://postgres:Gr@vitics3980@localhost/postgres')

In [ ]:
meta = MetaData(schema="countries")

In [ ]:
conn = engine.connect()

**Example(s) without pd.DataFrames - use fetchall**

In [ ]:
result = conn.execute("SELECT datname from pg_database")

In [ ]:
rows = result.fetchall()

In [ ]:
[x for x in rows]

In [ ]:
cities = conn.execute("select * \
from countries.countries \
inner join countries.cities \
on countries.cities.country_code = countries.code")

In [ ]:
cities_res = cities.fetchall()

In [ ]:
cities_list = [x for i, x in enumerate(cities_res) if i < 10]

In [ ]:
cities_list

# Introduction to joins

In this chapter, you'll be introduced to the concept of joining tables, and explore the different ways you can enrich your queries using inner joins and self-joins. You'll also see how to use the case statement to split up a field into different categories.

## Introduction to INNER JOIN

In [ ]:
cities = conn.execute("select * from countries.cities")

In [ ]:
cities_df = pd.read_sql("select * from countries.cities", conn)

In [ ]:
cities_df.head()

In [ ]:
sql_stmt = "SELECT * FROM countries.cities INNER JOIN countries.countries ON countries.cities.country_code = countries.countries.code"
pd.read_sql(sql_stmt, conn).head()

In [ ]:
sql_stmt = "SELECT countries.cities.name as city, countries.countries.name as country, \
countries.countries.region FROM countries.cities INNER JOIN countries.countries ON \
countries.cities.country_code = countries.countries.code"
pd.read_sql(sql_stmt, conn).head()

## INNER JOIN via USING

![](https://github.com/trenton3983/DataCamp/blob/master/Images/joining_data_in_sql/inner_join_diagram_small.JPG?raw=true)

```sql
SELECT left_table.id as L_id
       left_table.val as L_val
       right_table.val as R_val
FROM left_table
INNER JOIN right_table
ON left_table.id = right_table.id;
```

* When the key field you'd like to join on is the same name in both tables, you can use a `USING` clause instead of the `ON` clause.

```sql
SELECT left_table.id as L_id
       left_table.val as L_val
       right_table.val as R_val
FROM left_table
INNER JOIN right_table
USING (id);
```

### Countries with prime ministers and presidents

```sql
SELECT p1.country, p1.continent, prime_minister, president
FROM leaders.presidents AS p1
INNER JOIN leaders.prime_ministers as p2
USING (country);
```

In [ ]:
sql_stmt = "SELECT p1.country, p1.continent, prime_minister, president \
FROM leaders.presidents AS p1 \
INNER JOIN leaders.prime_ministers as p2 \
USING (country)"

pd.read_sql(sql_stmt, conn).head()

### Exercises

#### Review inner join using on

Why does the following code result in an error?

```sql
SELECT c.name AS country, l.name AS language
FROM countries AS c
  INNER JOIN languages AS l;
```

* `INNER JOIN` requires a specification of the key field (or fields) in each table.

#### Inner join with using

When joining tables with a common field name, e.g.

```sql
SELECT *
FROM countries
  INNER JOIN economies
    ON countries.code = economies.code
```

You can use `USING` as a shortcut:

```sql
SELECT *
FROM countries
  INNER JOIN economies
    USING(code)
```

You'll now explore how this can be done with the `countries` and `languages` tables.

**Instructions**

* Inner join `countries` on the left and `languages` on the right with `USING(code)`.
* Select the fields corresponding to:
    * country name `AS country`,
    * continent name,
    * language name `AS language`, and
    * whether or not the language is official.
* Remember to alias your tables using the first letter of their names.

```sql
-- 4. Select fields
SELECT c.name as country, c.continent, l.name as language, l.official
  -- 1. From countries (alias as c)
  FROM countries as c
  -- 2. Join to languages (as l)
  INNER JOIN languages as l
    -- 3. Match using code
    USING (code)
```

In [ ]:
sql_stmt = "SELECT c.name as country, c.continent, l.name as language, l.official \
FROM countries.countries as c \
INNER JOIN countries.languages as l \
USING (code)"

pd.read_sql(sql_stmt, conn).head()

## Self-ish joins, just in CASE

### self-join on prime_ministers

In [ ]:
sql_stmt = "SELECT * \
FROM leaders.prime_ministers"

pm_df = pd.read_sql(sql_stmt, conn)
pm_df.head()

* inner joins where a table is joined with itself
    * self join
* Explore how to slice a numerical field into categories using the CASE command
* Self-joins are used to compare values in a field to other values of the same field from within the same table
* Recall the prime ministers table:
    * What if you wanted to create a new table showing countries that are in the same continenet matched as pairs?

```sql
SELECT p1.country AS country1, p2.country AS country2, p1.continent
FROM leaders.prime_ministers as p1
INNER JOIN prime_ministers as p2
ON p1.continent = p2.continent;
```

In [ ]:
sql_stmt = "SELECT p1.country AS country1, p2.country AS country2, p1.continent \
FROM leaders.prime_ministers as p1 \
INNER JOIN leaders.prime_ministers as p2 \
ON p1.continent = p2.continent"

pm_df_1 = pd.read_sql(sql_stmt, conn)
pm_df_1.head()

* The country column is selected twice as well as continent. 
* The prime ministers table is on both the left and the right.
* The vital step is setting the key columns by which we match the table to itself.
    * For each country, there will be a match if the country in the "right table" (that's also prime_ministers) is in the same continent.
* This is a pairing of each country with every other country in its same continent
    * Conditions where country1 = country2 should not be included in the table

### Finishing off the self-join on prime_ministers

```sql
SELECT p1.country AS country1, p2.country AS country2, p1.continent
FROM leaders.prime_ministers as p1
INNER JOIN prime_ministers as p2
ON p1.continent = p2.continent AND p1.country <> p2.country;
```

In [ ]:
sql_stmt = "SELECT p1.country AS country1, p2.country AS country2, p1.continent \
FROM leaders.prime_ministers as p1 \
INNER JOIN leaders.prime_ministers as p2 \
ON p1.continent = p2.continent AND p1.country <> p2.country"

pm_df_2 = pd.read_sql(sql_stmt, conn)
pm_df_2.head()

In [ ]:
pm_df_1.equals(pm_df_2)

* `AND` clause can check that multiple conditions are met.
* Now a match will not be made between prime_minister and itself if the countries match

### CASE WHEN and THEN

* The states table contains numeric data about different countries in the six inhabited world continents
* Group the year of independence into categories of:
    * before 1900
    * between 1900 and 1930
    * and after 1930
* CASE is a way to do multiple if-then-else statements

```sql
SELECT name, continent, indep_year,
    CASE WHEN indep_year < 1900 THEN 'before 1900'
    WHEN indep_year <= 1930 THEN 'between 1900 and 1930'
    ELSE 'after 1930' END
    AS indep_year_group
FROM states
ORDER BY indep_year_group;
```

In [ ]:
sql_stmt = "SELECT name, continent, indep_year, \
CASE WHEN indep_year < 1900 THEN 'before 1900' \
WHEN indep_year <= 1930 THEN 'between 1900 and 1930' \
ELSE 'after 1930' END \
AS indep_year_group \
FROM leaders.states \
ORDER BY indep_year_group"

pd.read_sql(sql_stmt, conn)

### Exercises

#### Self-join

In this exercise, you'll use the `populations` table to perform a self-join to calculate the percentage increase in population from 2010 to 2015 for each country code!

Since you'll be joining the `populations` table to itself, you can alias `populations` as `p1` and also `populations` as `p2`. This is good practice whenever you are aliasing and your tables have the same first letter. Note that you are required to alias the tables with self-joins.

**Instructions 1/3**
* Join `populations` with itself `ON country_code`.
* Select the `country_code` from `p1` and the `size` field from both `p1` and `p2`. SQL won't allow same-named fields, so alias `p1.size as size2010` and `p2.size as size2015`.

```sql
-- 4. Select fields with aliases
SELECT p1.size as size2010,
    p1.country_code,
    p2.size as size2015
-- 1. From populations (alias as p1)
FROM countries.populations as p1
  -- 2. Join to itself (alias as p2)
    INNER JOIN countries.populations as p2
    -- 3. Match on country code
    ON p1.country_code = p2.country_code
```

In [ ]:
sql_stmt = "SELECT p1.size as size2010, p1.country_code, p2.size as size2015 \
FROM countries.populations as p1 \
INNER JOIN countries.populations as p2 \
ON p1.country_code = p2.country_code"

pd.read_sql(sql_stmt, conn).head()

**Instructions 2/3**

Notice from the result that for each country_code you have four entries laying out all combinations of 2010 and 2015.

* Extend the `ON` in your query to include only those records where the `p1.year` (2010) matches with `p2.year - 5` (2015 - 5 = 2010). This will omit the three entries per `country_code` that you aren't interested in.

```sql
-- 4. Select fields with aliases
SELECT p1.country_code,
       p1.size as size2010,
       p2.size as size2015
-- 1. From populations (alias as p1)
FROM countries.populations as p1
  -- 2. Join to itself (alias as p2)
    INNER JOIN countries.populations as p2
    -- 3. Match on country code
    ON p1.country_code = p2.country_code
      -- 4. and year (with calculation)
      AND p1.year = (p2.year - 5)
```

In [ ]:
sql_stmt = "SELECT p1.size as size2010, p1.country_code, p2.size as size2015 \
FROM countries.populations as p1 \
INNER JOIN countries.populations as p2 \
ON p1.country_code = p2.country_code \
AND p1.year = (p2.year - 5)"

pd.read_sql(sql_stmt, conn).head()

**Instructions 3/3**

As you just saw, you can also use SQL to calculate values like `p2.year - 5` for you. With two fields like `size2010` and `size2015`, you may want to determine the percentage increase from one field to the next:

With two numeric fields `A` and `B`, the percentage growth from `A` to `B` can be calculated as $$\frac{(B−A)}{A}∗100.0$$.

Add a new field to `SELECT`, aliased as `growth_perc`, that calculates the percentage population growth from 2010 to 2015 for each country, using `p2.size` and `p1.size`.


```sql
SELECT p1.country_code,
       p1.size AS size2010, 
       p2.size AS size2015,
       -- 1. calculate growth_perc
       ((p2.size - p1.size)/p1.size * 100.0) AS growth_perc
-- 2. From populations (alias as p1)
FROM countries.populations as p1
  -- 3. Join to itself (alias as p2)
  INNER JOIN countries.populations as p2
    -- 4. Match on country code
    ON p1.country_code = p2.country_code
        -- 5. and year (with calculation)
        AND p1.year = (p2.year - 5);
```

In [ ]:
sql_stmt = "SELECT p1.size as size2010, p1.country_code, p2.size as size2015, \
((p2.size - p1.size)/p1.size * 100.0) AS growth_perc \
FROM countries.populations as p1 \
INNER JOIN countries.populations as p2 \
ON p1.country_code = p2.country_code \
AND p1.year = (p2.year - 5)"

pd.read_sql(sql_stmt, conn).head()

#### Case when and then

Often it's useful to look at a numerical field not as raw data, but instead as being in different categories or groups.

You can use `CASE` with `WHEN`, `THEN`, `ELSE`, and `END` to define a new grouping field.

**Instructions**

Using the countries table, create a new field AS geosize_group that groups the countries into three groups:

* If `surface_area` is greater than 2 million, `geosize_group` is `'large'`.
* If `surface_area` is greater than 350 thousand but not larger than 2 million, `geosize_group` is `'medium'`.
* Otherwise, `geosize_group` is `'small'`.

```sql
SELECT name, continent, code, surface_area,
    -- 1. First case
    CASE WHEN surface_area > 2000000 THEN 'large'
        -- 2. Second case
        WHEN surface_area > 350000 THEN 'medium'
        -- 3. Else clause + end
        ELSE 'small' END
        -- 4. Alias name
        AS geosize_group
-- 5. From table
FROM countries.countries;
```

In [ ]:
sql_stmt = "SELECT name, continent, code, surface_area, \
CASE WHEN surface_area > 2000000 THEN 'large' \
WHEN surface_area > 350000 THEN 'medium' \
ELSE 'small' END \
AS geosize_group \
FROM countries.countries;"

pd.read_sql(sql_stmt, conn).head()

#### Inner challenge

The table you created with the added `geosize_group` field has been loaded for you here with the name `countries_plus`. Observe the use of (and the placement of) the `INTO` command to create this `countries_plus` table:

**If you have downloaded the data from DataCamp and already have a schema for countries, countries_plus is already one of the tables**

```sql
SELECT name, continent, code, surface_area,
    CASE WHEN surface_area > 2000000
            THEN 'large'
       WHEN surface_area > 350000
            THEN 'medium'
       ELSE 'small' END
       AS geosize_group
INTO countries_plus
FROM countries.countries;
```

You will now explore the relationship between the size of a country in terms of surface area and in terms of population using grouping fields created with `CASE`.

By the end of this exercise, you'll be writing two queries back-to-back in a single script. You got this!

**Instructions 1/3**

Using the `populations` table focused only for the `year` 2015, create a new field `AS popsize_group` to organize population `size` into

* `'large'` (> 50 million),
* `'medium'` (> 1 million), and
* `'small'` groups.

Select only the country code, population size, and this new `popsize_group` as fields.

```sql
SELECT country_code, size,
    -- 1. First case
    CASE WHEN size > 50000000 THEN 'large'
        -- 2. Second case
        WHEN size > 1000000 THEN 'medium'
        -- 3. Else clause + end
        ELSE 'small' END
        -- 4. Alias name
        AS popsize_group
-- 5. From table
FROM countries.populations
-- 6. Focus on 2015
WHERE year = 2015;
```

In [ ]:
sql_stmt = "\
SELECT country_code, size, \
CASE WHEN size > 50000000 THEN 'large' \
WHEN size > 1000000 THEN 'medium' \
ELSE 'small' END \
AS popsize_group \
FROM countries.populations \
WHERE year = 2015; \
"

pd.read_sql(sql_stmt, conn).head()

**Instructions 2/3**

* Use `INTO` to save the result of the previous query as `pop_plus`. You can see an example of this in the `countries_plus` code in the assignment text. Make sure to include a `;` at the end of your `WHERE` clause!
* Then, include another query below your first query to display all the records in `pop_plus` using `SELECT * FROM pop_plus`; so that you generate results and this will display `pop_plus` in query result.

**Execute the first part on the PostgreSQL schema to create pop_plus**
```sql
SELECT country_code, size,
    CASE WHEN size > 50000000 THEN 'large'
        WHEN size > 1000000 THEN 'medium'
        ELSE 'small' END
        AS popsize_group
-- 1. Into table        
INTO countries.pop_plus
FROM populations
WHERE year = 2015;
```

**Run this below**
```sql
-- 2. Select all columns of pop_plus
SELECT * FROM countries.pop_plus;
```

In [ ]:
sql_stmt = "\
SELECT * FROM countries.pop_plus; \
"

pd.read_sql(sql_stmt, conn).head()

**Instructions 3/3**

* Keep the first query intact that creates `pop_plus` using `INTO`.
* Write a query to join `countries_plus AS c` on the left with `pop_plus AS p` on the right matching on the country code fields.
* Sort the data based on `geosize_group`, in ascending order so that `large` appears on top.
* Select the `name`, `continent`, `geosize_group`, and `popsize_group` fields.

In [ ]:
sql_stmt = "\
SELECT c.name, c.continent, c.geosize_group, p.popsize_group \
FROM countries.countries_plus AS c \
INNER JOIN countries.pop_plus AS p \
ON c.code = p.country_code \
ORDER BY geosize_group ASC \
"

q_df = pd.read_sql(sql_stmt, conn)
q_df.head()

In [ ]:
q_df.tail()

# Outer JOINs and Cross JOINs

In this chapter, you'll come to grips with different kinds of outer joins. You'll learn how to gain further insights into your data through left joins, right joins, and full joins. In addition to outer joins, you'll also work with cross joins.

## LEFT and RIGHT JOINs

* You can remember outer joins as reaching out to another table while keeping all of the records of the original table.
* Inner joins keep only the records in both tables.
* This chapter will explore three types of OUTER JOINs:
    1. LEFT JOINs
    1. RIGHT JOINs
    1. FULL JOINs
* How a LEFT JOIN differs from an INNER JOIN:

**INNER JOIN**

![](https://github.com/trenton3983/DataCamp/blob/master/Images/joining_data_in_sql/inner_join_diagram_small.JPG?raw=true "INNER JOIN")
* The only records included in the resulting table of the INNER JOIN query were those in which the id field had matching values.

```sql
SELECT p1.country, prime_minister, president
FROM prime_ministers as p1
INNER JOIN presidents p2
ON p1.country = p2.country;
```

**LEFT JOIN**

![](https://github.com/trenton3983/DataCamp/blob/master/Images/joining_data_in_sql/left_join_diagram.JPG?raw=true "Left JOIN")
* In contrast, a LEFT JOIN notes those record in the left table that do not have a match on the key field in the right table.
* This is denoted in the diagram by the open circles remaining close to the left table for id values of 2 and 3.
* Whereas the INNER JOIN kept just the records corresponding to id values 1 and 44, a LEFT JOIN keeps all of the original records in the left table, but then marks the values as missing in the right table for those that don't have a match.
* The syntax of the LEFT JOIN is similar to that of the INNER JOIN.

```sql
SELECT p1.country, prime_minister, president
FROM prime_ministers as p1
LEFT JOIN presidents p2
ON p1.country = p2.country;
```

**LEFT JOIN multiple matches**

![](https://github.com/trenton3983/DataCamp/blob/master/Images/joining_data_in_sql/left_join_multi_diagram.JPG?raw=true "LEF JOIN multiple entries in right table")
* It isn't always the case that each key value in the left table corresponds to exactly one record in the key column of the right table.
* Duplicate rows are shown in the LEFT JOIN for id 1 since it has two matches corresponding to the values of R1 and R2 in the right2 table.

**RIGHT JOIN**

![](https://github.com/trenton3983/DataCamp/blob/master/Images/joining_data_in_sql/right_join_diagram.JPG?raw=true "RIGHT JOIN - uncommon")
* Instead of matching entries in the id column on the left table to the id column on the right table, a RIGHT JOIN does the reverse.
    * The resulting table from a RIGHT JOIN shows the missing entries in the L_val field.
* In the SQL statement the right table appears after RIGHT JOIN and the left table appears after FROM.

```sql
SELECT right_table.id AS R_id,
       left_table.val AS L_val,
       right_talbe.vale AS R_val
FROM left_table
RIGHT JOIN right_table
ON left_table.id = right_table.id;
```

#### INNER JOIN

In [ ]:
sql_stmt = "\
SELECT p1.country, prime_minister, president \
FROM leaders.prime_ministers as p1 \
INNER JOIN leaders.presidents as p2 \
ON p1.country = p2.country \
"

pd.read_sql(sql_stmt, conn)

#### LEFT JOIN

* The first four records are the same as those from INNER JOIN
* The following records correspond to the countries that do not have a president and thus their president values are missing.

In [ ]:
sql_stmt = "\
SELECT p1.country, prime_minister, president \
FROM leaders.prime_ministers as p1 \
LEFT JOIN leaders.presidents as p2 \
ON p1.country = p2.country \
"

pd.read_sql(sql_stmt, conn)

### Exercises

#### LEFT JOIN

Now you'll explore the differences between performing an inner join and a left join using the `cities` and `countries` tables.

You'll begin by performing an inner join with the `cities` table on the left and the `countries` table on the right. Remember to alias the name of the city field as `city` and the name of the country field as `country`.

You will then change the query to a left join. Take note of how many records are in each query here!

**Instructions 1/2**

* Fill in the code based on the instructions in the code comments to complete the inner join. Note how many records are in the result of the join in the **query result** tab.

```sql
-- Select the city name (with alias), the country code,
-- the country name (with alias), the region,
-- and the city proper population
SELECT c1.name AS city, code, c2.name AS country,
       region, city_proper_pop
-- From left table (with alias)
FROM cities AS c1
  -- Join to right table (with alias)
  INNER JOIN countries AS c2
    -- Match on country code
    ON c1.country_code = c2.code
-- Order by descending country code
ORDER BY code DESC;
```

In [ ]:
sql_stmt = "\
SELECT c1.name AS city, code, c2.name AS country, region, city_proper_pop \
FROM countries.cities AS c1 \
INNER JOIN countries.countries AS c2 \
ON c1.country_code = c2.code \
ORDER BY code DESC; \
"

pd.read_sql(sql_stmt, conn).head()

**Instructions 2/2**

* Change the code to perform a `LEFT JOIN` instead of an `INNER JOIN`. After executing this query, note how many records the query result contains.

```sql
SELECT c1.name AS city, code, c2.name AS country,
       region, city_proper_pop
FROM cities AS c1
  -- 1. Join right table (with alias)
  LEFT JOIN countries AS c2
    -- 2. Match on country code
    ON c1.country_code = c2.code
-- 3. Order by descending country code
ORDER BY code DESC;
```

In [ ]:
sql_stmt = "\
SELECT c1.name AS city, code, c2.name AS country, region, city_proper_pop \
FROM countries.cities AS c1 \
LEFT JOIN countries.countries AS c2 \
ON c1.country_code = c2.code \
ORDER BY code DESC; \
"

pd.read_sql(sql_stmt, conn).head()

#### JEFT JOIN (2)

##### LEFT JOIN (3)

#### RIGHT JOIN

## FULL JOINs

### Exercises

#### FULL JOIN

#### FULL JOIN (2)

#### FULL JOIN (3)

#### Review OUTER JOINs

## CROSSing the rubicon

#### Exercises

##### A table of two cities

##### Outer challenge

# Set theory clauses

In this chapter, you'll learn more about set theory using Venn diagrams and you will be introduced to union, union all, intersect, and except clauses. You'll finish by investigating semi-joins and anti-joins, which provide a nice introduction to subqueries.

## State of the UNION

### Exercises

#### UNION

#### UNION (2)

#### UNION ALL

## INTERSECTional data science

### Exercises

#### INTERSECT

#### INTERSECT (2)

#### Review UNION and INTERSECT

## EXCEPTional

### Exercises

#### EXCEPT

#### EXCEPT (2)

## Semi-JOINs and Anti-JOINS

### Exercises

#### Semi-JOIN

#### Relating semi-JOIN to a tweaked INNER JOIN

#### Diagnosing problems using anti-JOIN

#### Set theory challenge

# Subqueries

In this closing chapter, you'll learn how to use nested queries to add some finesse to your data insights. You'll also wrap all of the content covered throughout this course into solving three challenge problems.

## Subqueries inside WHERE and SELECT clauses

### Exercises

#### Subquery inside WHERE

#### Subquery inside WHERE (2)

#### Subquery inside SELECT

## Subquery inside FROM clause

### Exercises

#### Subquery inside FROM

#### Advanced subquery

#### Subquery challenge

#### Subquery review

## Course Review

### Final Challenge

### Final Challenge (2)

### Final Challenge (3)

# Trimet

![](https://github.com/trenton3983/DataCamp/blob/master/Images/joining_data_in_sql/postgres_trimet.JPG?raw=true)

In [ ]:
sql_stmt = "SELECT * FROM trimet.route"

In [ ]:
trimet_route = pd.read_sql(sql_stmt, conn)

In [ ]:
trimet_route

In [ ]:
sql_stmt = "SELECT * FROM trimet.agency"

In [ ]:
trimet_agency = pd.read_sql(sql_stmt, conn)

In [ ]:
trimet_agency

In [ ]:
sql_stmt = "select tr.route_number, ta.agency_name \
from trimet.route tr \
inner join trimet.agency ta \
on tr.agency_id = ta.id where agency_name = 'Trimet'"

In [ ]:
pd.read_sql(sql_stmt, conn)

In [ ]:
conn.close()